In [1]:
import numpy as np
import h5py
from scipy.io import loadmat

In [2]:
def Patch(data,height_index,width_index,PATCH_SIZE):   # PATCH_SIZE为一个patch（边长-1）的一半    data维度(H,W,C)
    height_slice = slice(height_index-PATCH_SIZE, height_index+PATCH_SIZE)
    width_slice = slice(width_index-PATCH_SIZE, width_index+PATCH_SIZE)
    # 由height_index和width_index定位patch中心像素
    patch = data[height_slice, width_slice,:]
    patch = patch.reshape(-1,patch.shape[0]*patch.shape[1]*patch.shape[2])
    # print(patch.shape)
    return patch

In [3]:
img = loadmat('../../dataset/2_Botswana/Botswana.mat')['Botswana']
print(img.shape)
gt = loadmat('../../dataset/2_Botswana/Botswana_gt')['Botswana_gt']
print(gt.shape)

'''
(1476, 256, 145)
(1476, 256)
'''

(1476, 256, 145)
(1476, 256)


'\n(1476, 256, 145)\n(1476, 256)\n'

In [4]:
dict_k = {}
for i in range(gt.shape[0]):
    for j in range(gt.shape[1]):
        if gt[i][j] in range(0, 16):
            if gt[i][j] not in dict_k:
                dict_k[gt[i][j]] = 0
            dict_k[gt[i][j]] += 1

print(dict_k)  

'''
{0: 374608, 14: 95, 12: 181, 13: 268, 11: 305, 10: 248, 9: 314, 7: 259, 1: 270, 2: 101, 5: 269, 8: 203, 6: 269, 3: 251, 4: 215}
'''

{0: 374608, 14: 95, 12: 181, 13: 268, 11: 305, 10: 248, 9: 314, 7: 259, 1: 270, 2: 101, 5: 269, 8: 203, 6: 269, 3: 251, 4: 215}


'\n{0: 374608, 14: 95, 12: 181, 13: 268, 11: 305, 10: 248, 9: 314, 7: 259, 1: 270, 2: 101, 5: 269, 8: 203, 6: 269, 3: 251, 4: 215}\n'

In [5]:
'''
波段处理
'''
bands_select = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103]
for i in range(len(bands_select)):
    bands_select[i] = bands_select[i] - 1 
# print(bands_select)

'''
波段选择
'''
img = img[:, :, bands_select]
print(img.shape)

'''
图像归一化
'''
img = ( img * 1.0 - img.min() ) / ( img.max() - img.min() )
type(img)

(1476, 256, 100)


numpy.ndarray

In [6]:
[m, n, b] = img.shape
label_num = gt.max()  
PATCH_SIZE = 14
print(label_num)
'''
最大为14，即除0外包括14类
'''

'''

'''

14


'\n\n'

In [7]:
# temp_label = np.zeros((1, label_num), dtype=np.int8) 
# temp_label, temp_label.shape

In [8]:
# padding the hyperspectral images
img_temp = np.zeros((m + 2 * PATCH_SIZE, n + 2 * PATCH_SIZE, b), dtype=np.float32)
img_temp[PATCH_SIZE:(m + PATCH_SIZE), PATCH_SIZE:(n + PATCH_SIZE), :] = img[:, :, :]

for i in range(PATCH_SIZE):
    img_temp[i, :, :] = img_temp[2 * PATCH_SIZE - i, :, :]
    img_temp[m + PATCH_SIZE + i, :, :] = img_temp[m + PATCH_SIZE - i - 2, :, :]

for i in range(PATCH_SIZE):
    img_temp[:, i, :] = img_temp[:, 2 * PATCH_SIZE - i, :]
    img_temp[:, n + PATCH_SIZE + i, :] = img_temp[:, n + PATCH_SIZE  - i - 2, :]

img = img_temp

gt_temp = np.zeros((m + 2 * PATCH_SIZE, n + 2 * PATCH_SIZE), dtype=np.int8)
gt_temp[PATCH_SIZE:(m + PATCH_SIZE), PATCH_SIZE:(n + PATCH_SIZE)] = gt[:, :]
gt = gt_temp

print(img.shape)
print(gt.shape)

'''
(1504, 284, 100)
(1504, 284)
'''

(1504, 284, 100)
(1504, 284)


'\n(1504, 284, 100)\n(1504, 284)\n'

In [9]:
[m, n, b] = img.shape
data = []
label = []
count = 0

In [10]:
m, n ,b

(1504, 284, 100)

In [11]:
for i in range(PATCH_SIZE, m - PATCH_SIZE):  # 没有patch的那一部分，也就是原来的图像
    for j in range(PATCH_SIZE, n - PATCH_SIZE):
        if gt[i, j] == 0:
            continue
        else:
            count += 1
            temp_data = Patch(img, i, j, PATCH_SIZE)
            temp_label = gt[i, j] - 1

            data.append(temp_data)  # 每一行表示一个patch
            label.append(temp_label)
print(count)

'''
3248
'''

3248


'\n3248\n'

In [13]:
data = np.array(data)
print(data.shape)
''' (3248, 1, 78400) '''

data = np.squeeze(data)
print("squeeze : ", data.shape)
''' squeeze :  (3248, 78400) '''


label = np.array(label)
print(label.shape)
''' (3248,) '''

label = np.squeeze(label)
print("squeeze : ", label.shape)
''' squeeze :  (3248,) '''


np.set_printoptions(threshold=np.inf)

print(label)

(3248, 78400)
squeeze :  (3248, 78400)
(3248,)
squeeze :  (3248,)
[13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13
 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13
 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13
 13 13 13 13 13 13 13 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11
 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11
 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11
 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11
 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11
 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11
 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11
 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 12 12 12 12
 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12
 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 

In [22]:
print(type(label))
bot_dict = {}
bot_dict = bot_dict.fromkeys([i for i in range(label_num)], 0)
print(bot_dict)

for i in label:
    if i in bot_dict:
        bot_dict[i] += 1
print(bot_dict)
bot_label_sum = sum(bot_dict.values())
print(bot_label_sum)

# <class 'numpy.ndarray'>
# {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0}
# {0: 270, 1: 101, 2: 251, 3: 215, 4: 269, 5: 269, 6: 259, 7: 203, 8: 314, 9: 248, 10: 305, 11: 181, 12: 268, 13: 95}
# 3248

<class 'numpy.ndarray'>
{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0}
{0: 270, 1: 101, 2: 251, 3: 215, 4: 269, 5: 269, 6: 259, 7: 203, 8: 314, 9: 248, 10: 305, 11: 181, 12: 268, 13: 95}
3248


In [13]:
# np.set_printoptions(threshold=np.inf)
# print(type(label))
# print(label)

In [14]:
indices = np.arange(data.shape[0]) 
# (3248,)
shuffled_indices = np.random.permutation(indices)
# (3248,)
images = data[shuffled_indices]
# (3248, 78400)
labels = label[shuffled_indices]
# (3248,)

In [15]:
t_labeled = []

dict_sample = \
{'0': 374608, '14': 95, '12': 181, '13': 268, '11': 305, '10': 248, '9': 314, 
 '7': 259, '1': 270, '2': 101, '5': 269, '8': 203, '6': 269, '3': 251, '4': 215}

for c in range(label_num): 
    if dict_sample[str(c+1)]<200:   # ''' 标记样本数小于200的不要'''
       pass
    else:
        i = indices[labels == c][:200]
        t_labeled += list(i)

In [16]:
t_images = images[t_labeled]
print('t_images', t_images.shape)
t_labels = labels[t_labeled]
print('t_labels', t_labels.shape)

'''
t_images (2200, 78400)
t_labels (2200,)
'''

t_images (2200, 78400)
t_labels (2200,)


'\nt_images (2200, 78400)\nt_labels (2200,)\n'

In [17]:
f = h5py.File(r'../../h5dataset_ica_bandselect/BO-' + str(PATCH_SIZE * 2) + '-' + str(PATCH_SIZE * 2) + '-100.h5', 'w')  # 每类200个
f['data'] = images
f['label'] = labels
f.close()

f = h5py.File(r'../../h5dataset_ica_bandselect_200/BO-' + str(PATCH_SIZE * 2) + '-' + str(PATCH_SIZE * 2) + '-100.h5', 'w')  # 每类200个
f['data'] = t_images
f['label'] = t_labels
f.close()

print("success")

success


In [18]:
#HDF5的读取
f = h5py.File('../../h5dataset_ica_bandselect/BO-28-28-100.h5','r')   #打开h5文件  # 可以查看所有的主键  
# print(type(f))

print([key for key in f.keys()])

# print('first, we get values of x:', f['data'][:])
# print('then, we get values of y:', f['label'][:])
print(f['data'][:].shape)
print(f['label'][:].shape)

['data', 'label']
(3248, 78400)
(3248,)


In [19]:
#HDF5的读取
f = h5py.File('../../h5dataset_ica_bandselect_200/BO-28-28-100.h5','r')   #打开h5文件  # 可以查看所有的主键  
# print(type(f))

print([key for key in f.keys()])

# print('first, we get values of x:', f['data'][:])
# print('then, we get values of y:', f['label'][:])
print(f['data'][:].shape)
print(f['label'][:].shape)

['data', 'label']
(2200, 78400)
(2200,)


In [20]:
#HDF5的读取
f = h5py.File('../../h5dataset/BO-28-28-100.h5','r')   #打开h5文件  # 可以查看所有的主键  
# print(type(f))

print([key for key in f.keys()])

# print('first, we get values of x:', f['data'][:])
# print('then, we get values of y:', f['label'][:])
print(f['data'][:].shape)
print(f['label'][:].shape)

['data', 'label']
(2200, 78400)
(2200,)
